# NeurIPS Open Polymer Prediction 2025 - Complete Solution

This notebook contains the complete solution for the NeurIPS Open Polymer Prediction 2025 challenge.

## Challenge Overview

- **Task**: Predict 5 polymer properties from SMILES strings
- **Properties**: Tg (glass transition temperature), FFV (fractional free volume), Tc (thermal conductivity), Density, Rg (radius of gyration)
- **Evaluation**: Weighted Mean Absolute Error (wMAE)
- **Data**: 7,973 training samples with significant missing values

## Table of Contents
1. [Setup and Imports](#setup)
2. [Data Loading and Exploration](#data)
3. [Molecular Featurization](#featurization)
4. [Dataset Implementation](#dataset)
5. [Model Architecture](#model)
6. [Training Implementation](#training)
7. [Competition Metrics](#metrics)
8. [Training Loop](#training-loop)
9. [Evaluation and Submission](#submission)
10. [Results and Analysis](#results)

## 1. Setup and Imports

First, let's install the required packages and import all necessary libraries.

In [ ]:
# Install required packages (run this if packages are not installed)
# !pip install torch torch-geometric rdkit pandas numpy scikit-learn tqdm matplotlib seaborn

In [ ]:
# Core imports
import os
import sys
import random
import warnings
from datetime import datetime
from typing import List, Dict, Tuple, Optional

# Data handling
import pandas as pd
import numpy as np
from tqdm import tqdm

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# PyTorch and PyTorch Geometric
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import random_split
from torch_geometric.data import Data, Dataset, DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool

# RDKit for chemistry
from rdkit import Chem
from rdkit.Chem import rdchem
from rdkit import RDLogger

# Suppress RDKit warnings
RDLogger.DisableLog('rdApp.*')
warnings.filterwarnings('ignore')

# Set style for plots
plt.style.use('default')
sns.set_palette("husl")

print("All imports successful!")
print(f"PyTorch version: {torch.__version__}")
print(f"Device available: {'CUDA' if torch.cuda.is_available() else 'CPU'}")

In [ ]:
# Configuration
class Config:
    """Configuration class for model hyperparameters and settings."""
    
    def __init__(self):
        # Device
        self.DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        # Model hyperparameters
        self.NUM_EPOCHS = 50
        self.BATCH_SIZE = 32
        self.LEARNING_RATE = 1e-3
        self.WEIGHT_DECAY = 1e-5
        self.HIDDEN_CHANNELS = 128
        self.NUM_GCN_LAYERS = 3
        
        # Data
        self.VAL_SPLIT_FRACTION = 0.2
        self.SEED = 42
        
        # Target properties
        self.TARGET_PROPERTIES = ['Tg', 'FFV', 'Tc', 'Density', 'Rg']

# Create configuration instance
CONFIG = Config()

print(f"Configuration:")
print(f"  Device: {CONFIG.DEVICE}")
print(f"  Batch size: {CONFIG.BATCH_SIZE}")
print(f"  Learning rate: {CONFIG.LEARNING_RATE}")
print(f"  Hidden channels: {CONFIG.HIDDEN_CHANNELS}")
print(f"  Target properties: {CONFIG.TARGET_PROPERTIES}")

In [ ]:
# Set random seeds for reproducibility
def set_seed(seed: int = 42):
    """Set random seeds for reproducibility."""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed(CONFIG.SEED)
print(f"Random seed set to {CONFIG.SEED}")

## 2. Data Loading and Exploration

Let's load the competition data and explore its structure.

In [ ]:
# Load competition data
def load_competition_data():
    """Load the competition data files."""
    train_path = "info/train.csv"
    test_path = "info/test.csv"
    
    if not os.path.exists(train_path):
        raise FileNotFoundError(f"Training data not found at {train_path}")
    if not os.path.exists(test_path):
        raise FileNotFoundError(f"Test data not found at {test_path}")
    
    train_df = pd.read_csv(train_path)
    test_df = pd.read_csv(test_path)
    
    print(f"Loaded training data: {len(train_df)} samples")
    print(f"Loaded test data: {len(test_df)} samples")
    
    return train_df, test_df

# Load data
train_df, test_df = load_competition_data()

# Display basic info
print("\nTraining data columns:", train_df.columns.tolist())
print("Training data shape:", train_df.shape)
print("Test data shape:", test_df.shape)

In [ ]:
# Explore the data
print("First few rows of training data:")
display(train_df.head())

print("\nFirst few rows of test data:")
display(test_df.head())

In [ ]:
# Analyze missing values
print("Missing values analysis:")
print("=" * 50)

missing_stats = []
for col in CONFIG.TARGET_PROPERTIES:
    if col in train_df.columns:
        missing = train_df[col].isna().sum()
        total = len(train_df)
        percentage = (missing / total) * 100
        missing_stats.append({
            'Property': col,
            'Missing': missing,
            'Total': total,
            'Percentage': percentage
        })
        print(f"{col:>8}: {missing:>5}/{total} ({percentage:>5.1f}%)")

# Create a DataFrame for visualization
missing_df = pd.DataFrame(missing_stats)

# Visualize missing values
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
sns.barplot(data=missing_df, x='Property', y='Percentage')
plt.title('Missing Values by Property')
plt.ylabel('Missing Percentage (%)')
plt.xticks(rotation=45)

plt.subplot(1, 2, 2)
# Heatmap of missing values pattern
missing_matrix = train_df[CONFIG.TARGET_PROPERTIES].isna().astype(int)
sns.heatmap(missing_matrix.iloc[:100], cmap='RdYlBu_r', cbar_kws={'label': 'Missing (1) / Present (0)'})
plt.title('Missing Values Pattern (First 100 samples)')
plt.xlabel('Properties')
plt.ylabel('Samples')

plt.tight_layout()
plt.show()

In [ ]:
# Analyze property distributions
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, prop in enumerate(CONFIG.TARGET_PROPERTIES):
    if prop in train_df.columns:
        # Remove missing values for plotting
        values = train_df[prop].dropna()
        
        axes[i].hist(values, bins=50, alpha=0.7, edgecolor='black')
        axes[i].set_title(f'{prop} Distribution\n(n={len(values)}, mean={values.mean():.3f})')
        axes[i].set_xlabel(prop)
        axes[i].set_ylabel('Frequency')
        axes[i].grid(True, alpha=0.3)

# Remove empty subplot
fig.delaxes(axes[5])
plt.tight_layout()
plt.show()

# Summary statistics
print("\nSummary statistics for target properties:")
display(train_df[CONFIG.TARGET_PROPERTIES].describe())

## 3. Molecular Featurization

Convert SMILES strings to graph representations using RDKit.

In [ ]:
def get_atom_features(atom):
    """Extract features for a single atom."""
    # Basic atom features
    features = [
        atom.GetAtomicNum(),
        atom.GetDegree(),
        atom.GetTotalNumHs(),
        atom.GetTotalValence(),
        int(atom.GetIsAromatic()),
        int(atom.GetChiralTag()),
        atom.GetFormalCharge(),
        int(atom.IsInRing()),
    ]
    
    # One-hot encode common atomic numbers in polymers
    common_atoms = [1, 6, 7, 8, 9, 14, 15, 16, 17, 35, 53]  # H, C, N, O, F, Si, P, S, Cl, Br, I
    atomic_num_one_hot = [0] * (len(common_atoms) + 1)  # +1 for "other"
    
    atomic_num = atom.GetAtomicNum()
    if atomic_num in common_atoms:
        atomic_num_one_hot[common_atoms.index(atomic_num)] = 1
    else:
        atomic_num_one_hot[-1] = 1  # "other" category
    
    # Hybridization one-hot encoding
    hybridization_types = [
        rdchem.HybridizationType.SP,
        rdchem.HybridizationType.SP2,
        rdchem.HybridizationType.SP3,
        rdchem.HybridizationType.SP3D,
        rdchem.HybridizationType.SP3D2,
    ]
    hybridization_one_hot = [0] * (len(hybridization_types) + 1)  # +1 for "other"
    
    hybridization = atom.GetHybridization()
    if hybridization in hybridization_types:
        hybridization_one_hot[hybridization_types.index(hybridization)] = 1
    else:
        hybridization_one_hot[-1] = 1
    
    return features + atomic_num_one_hot + hybridization_one_hot


def get_bond_features(bond):
    """Extract features for a single bond."""
    # Bond type one-hot encoding
    bond_types = [
        Chem.rdchem.BondType.SINGLE,
        Chem.rdchem.BondType.DOUBLE,
        Chem.rdchem.BondType.TRIPLE,
        Chem.rdchem.BondType.AROMATIC,
    ]
    bond_type_one_hot = [0] * (len(bond_types) + 1)  # +1 for "other"
    
    bond_type = bond.GetBondType()
    if bond_type in bond_types:
        bond_type_one_hot[bond_types.index(bond_type)] = 1
    else:
        bond_type_one_hot[-1] = 1
    
    # Additional bond features
    features = [
        int(bond.IsInRing()),
        int(bond.GetIsConjugated()),
    ]
    
    return features + bond_type_one_hot


def smiles_to_graph(smiles_string: str):
    """Convert a SMILES string to a PyG Data object."""
    mol = Chem.MolFromSmiles(smiles_string)
    if mol is None:
        return None
    
    mol = Chem.AddHs(mol)  # Add explicit hydrogens

    # Get atom features
    atom_features = [get_atom_features(atom) for atom in mol.GetAtoms()]
    x = torch.tensor(atom_features, dtype=torch.float)

    # Get bond features and connectivity
    if mol.GetNumBonds() > 0:
        edge_indices = []
        edge_attrs = []
        for bond in mol.GetBonds():
            i = bond.GetBeginAtomIdx()
            j = bond.GetEndAtomIdx()
            edge_indices.append((i, j))
            edge_indices.append((j, i))  # Graph must be undirected

            # Enhanced bond features
            bond_features = get_bond_features(bond)
            edge_attrs.append(bond_features)
            edge_attrs.append(bond_features)  # Same features for both directions

        edge_index = torch.tensor(edge_indices, dtype=torch.long).t().contiguous()
        edge_attr = torch.tensor(edge_attrs, dtype=torch.float)
    else:
        # Handle molecules with no bonds (e.g., single atoms)
        edge_index = torch.empty((2, 0), dtype=torch.long)
        edge_attr = torch.empty((0, 7), dtype=torch.float)  # 7 bond features

    # Create the PyG Data object
    data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr)
    
    # Store number of features for model instantiation later
    data.num_atom_features = x.size(1)

    return data

print("Molecular featurization functions defined!")
print(f"Expected atom feature size: {len(get_atom_features(Chem.MolFromSmiles('C').GetAtomWithIdx(0)))}")

In [ ]:
# Test featurization on a sample SMILES
sample_smiles = train_df['SMILES'].iloc[0]
print(f"Sample SMILES: {sample_smiles}")

sample_graph = smiles_to_graph(sample_smiles)
if sample_graph is not None:
    print(f"Graph created successfully!")
    print(f"  Number of atoms: {sample_graph.x.size(0)}")
    print(f"  Atom feature size: {sample_graph.x.size(1)}")
    print(f"  Number of bonds: {sample_graph.edge_index.size(1) // 2}")
    print(f"  Edge feature size: {sample_graph.edge_attr.size(1) if sample_graph.edge_attr.size(0) > 0 else 0}")
else:
    print("Failed to create graph from SMILES")

# Test on a few more samples to check for failures
print("\nTesting featurization on first 10 SMILES...")
success_count = 0
for i, smiles in enumerate(train_df['SMILES'].head(10)):
    graph = smiles_to_graph(smiles)
    if graph is not None:
        success_count += 1
    else:
        print(f"Failed on sample {i}: {smiles}")

print(f"Success rate: {success_count}/10")

## 4. Dataset Implementation

Create a PyTorch Geometric dataset that handles multi-target prediction and missing values.

In [ ]:
class PolymerDataset(Dataset):
    """PyTorch Geometric Dataset for polymer data with multi-target prediction."""

    def __init__(self, df: pd.DataFrame, target_cols: Optional[List[str]] = None, is_test: bool = False):
        super().__init__()
        self.df = df
        self.is_test = is_test
        
        # Default target columns for the competition
        if target_cols is None:
            target_cols = CONFIG.TARGET_PROPERTIES
        self.target_cols = target_cols
        
        self.smiles_list = df['SMILES'].tolist()
        self.ids = df['id'].tolist()
        
        if not is_test:
            # Extract targets and create masks for missing values
            self.targets = []
            self.masks = []
            
            for idx in range(len(df)):
                target_values = []
                mask_values = []
                
                for col in target_cols:
                    if col in df.columns:
                        val = df.iloc[idx][col]
                        if pd.isna(val):
                            target_values.append(0.0)  # Placeholder for missing values
                            mask_values.append(0.0)    # Mask indicates missing
                        else:
                            target_values.append(float(val))
                            mask_values.append(1.0)    # Mask indicates present
                    else:
                        target_values.append(0.0)
                        mask_values.append(0.0)
                
                self.targets.append(target_values)
                self.masks.append(mask_values)
        
        self.cache = {}  # Cache graphs to avoid re-computing

    def len(self):
        return len(self.df)

    def get(self, idx):
        if idx in self.cache:
            data = self.cache[idx]
        else:
            smiles = self.smiles_list[idx]
            data = smiles_to_graph(smiles)
            if data is None:  # Handle RDKit parsing errors
                return None
            self.cache[idx] = data

        # Add ID (store as integer, not tensor)
        data.id = int(self.ids[idx])
        
        if not self.is_test:
            # Add target values and masks - reshape to (1, 5) to preserve structure during batching
            data.y = torch.tensor(self.targets[idx], dtype=torch.float).unsqueeze(0)  # Shape: (1, 5)
            data.mask = torch.tensor(self.masks[idx], dtype=torch.float).unsqueeze(0)  # Shape: (1, 5)
        
        return data

print("PolymerDataset class defined!")

In [ ]:
# Test the dataset
print("Testing dataset creation...")

# Create a small test dataset
test_dataset = PolymerDataset(train_df.head(10), is_test=False)
print(f"Dataset length: {len(test_dataset)}")

# Test a single sample
sample = test_dataset.get(0)
if sample is not None:
    print(f"Sample successful:")
    print(f"  ID: {sample.id}")
    print(f"  Node features shape: {sample.x.shape}")
    print(f"  Edge index shape: {sample.edge_index.shape}")
    print(f"  Targets shape: {sample.y.shape}")
    print(f"  Mask shape: {sample.mask.shape}")
    print(f"  Targets: {sample.y.squeeze()}")
    print(f"  Mask: {sample.mask.squeeze()}")
else:
    print("Sample failed")

# Test batching
print("\nTesting data loader...")
loader = DataLoader(test_dataset, batch_size=4, shuffle=False)
for batch in loader:
    print(f"Batch:")
    print(f"  Batch size: {batch.num_graphs}")
    print(f"  Node features shape: {batch.x.shape}")
    print(f"  Targets shape: {batch.y.shape}")
    print(f"  Mask shape: {batch.mask.shape}")
    break

## 5. Model Architecture

Implement a Graph Convolutional Network for multi-target polymer property prediction.

In [ ]:
class PolymerGCN(nn.Module):
    """Graph Convolutional Network for polymer property prediction."""

    def __init__(self, num_atom_features: int, hidden_channels: int, num_gcn_layers: int):
        super().__init__()
        self.convs = nn.ModuleList()
        self.bns = nn.ModuleList()

        # Input layer
        self.convs.append(GCNConv(num_atom_features, hidden_channels))
        self.bns.append(nn.BatchNorm1d(hidden_channels))

        # Hidden layers
        for _ in range(num_gcn_layers - 1):
            self.convs.append(GCNConv(hidden_channels, hidden_channels))
            self.bns.append(nn.BatchNorm1d(hidden_channels))

        # Output layer for multi-target prediction (5 properties)
        self.out = nn.Sequential(
            nn.Linear(hidden_channels, hidden_channels // 2),
            nn.ReLU(),
            nn.Linear(hidden_channels // 2, 5)  # 5 properties: Tg, FFV, Tc, Density, Rg
        )

    def forward(self, data):
        """Forward pass through the network."""
        x, edge_index, batch = data.x, data.edge_index, data.batch
        
        # Message passing through GCN layers
        for conv, bn in zip(self.convs, self.bns):
            x = conv(x, edge_index)
            x = bn(x)
            x = F.relu(x)

        # Readout phase: Aggregate node features into a single graph-level representation
        x = global_mean_pool(x, batch)
        
        # Final prediction
        return self.out(x)

print("PolymerGCN model class defined!")

In [ ]:
# Test model creation
print("Testing model creation...")

# Get feature dimensions from a sample
sample_data = test_dataset.get(0)
num_atom_features = sample_data.num_atom_features

# Create model
model = PolymerGCN(
    num_atom_features=num_atom_features,
    hidden_channels=CONFIG.HIDDEN_CHANNELS,
    num_gcn_layers=CONFIG.NUM_GCN_LAYERS
).to(CONFIG.DEVICE)

print(f"Model created successfully!")
print(f"  Input features: {num_atom_features}")
print(f"  Hidden channels: {CONFIG.HIDDEN_CHANNELS}")
print(f"  GCN layers: {CONFIG.NUM_GCN_LAYERS}")
print(f"  Output targets: 5")
print(f"  Device: {CONFIG.DEVICE}")

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"  Total parameters: {total_params:,}")
print(f"  Trainable parameters: {trainable_params:,}")

# Test forward pass
model.eval()
with torch.no_grad():
    # Create a small batch
    loader = DataLoader(test_dataset, batch_size=2, shuffle=False)
    batch = next(iter(loader))
    batch = batch.to(CONFIG.DEVICE)
    
    output = model(batch)
    print(f"\nForward pass test:")
    print(f"  Input batch size: {batch.num_graphs}")
    print(f"  Output shape: {output.shape}")
    print(f"  Output sample: {output[0].cpu().numpy()}")

## 6. Training Implementation

Implement training functions that handle missing values and multi-target prediction.

In [ ]:
def masked_mse_loss(predictions, targets, masks):
    """Calculate MSE loss only for non-missing values."""
    # Ensure all tensors have the same shape
    assert predictions.shape == targets.shape == masks.shape, f"Shape mismatch: pred {predictions.shape}, target {targets.shape}, mask {masks.shape}"
    
    # Only compute loss for non-missing values
    masked_predictions = predictions * masks
    masked_targets = targets * masks
    
    # Calculate squared differences
    squared_diff = (masked_predictions - masked_targets) ** 2
    
    # Sum over all dimensions and divide by number of non-missing values
    total_loss = torch.sum(squared_diff)
    total_count = torch.sum(masks)
    
    if total_count > 0:
        return total_loss / total_count
    else:
        return torch.tensor(0.0, device=predictions.device)


def train_one_epoch(model, loader, optimizer, device):
    """Perform one full training pass over the dataset."""
    model.train()
    total_loss = 0
    total_samples = 0
    
    for data in tqdm(loader, desc="Training", leave=False):
        data = data.to(device)
        optimizer.zero_grad()
        
        out = model(data)
        loss = masked_mse_loss(out, data.y, data.mask)
        
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item() * data.num_graphs
        total_samples += data.num_graphs
        
    return total_loss / total_samples


@torch.no_grad()
def evaluate(model, loader, device):
    """Evaluate the model on a dataset."""
    model.eval()
    total_loss = 0
    total_samples = 0
    
    all_preds = []
    all_targets = []
    all_masks = []
    
    for data in tqdm(loader, desc="Evaluating", leave=False):
        data = data.to(device)
        out = model(data)
        
        loss = masked_mse_loss(out, data.y, data.mask)
        total_loss += loss.item() * data.num_graphs
        total_samples += data.num_graphs
        
        all_preds.append(out.cpu())
        all_targets.append(data.y.cpu())
        all_masks.append(data.mask.cpu())

    avg_loss = total_loss / total_samples
    
    # Calculate per-property RMSE
    preds = torch.cat(all_preds, dim=0)  # Shape: (N, 5)
    targets = torch.cat(all_targets, dim=0)  # Shape: (N, 5)
    masks = torch.cat(all_masks, dim=0)  # Shape: (N, 5)
    
    property_names = CONFIG.TARGET_PROPERTIES
    rmses = {}
    
    for i, prop_name in enumerate(property_names):
        prop_mask = masks[:, i]
        if prop_mask.sum() > 0:  # Only calculate if we have non-missing values
            prop_preds = preds[:, i][prop_mask == 1]
            prop_targets = targets[:, i][prop_mask == 1]
            rmse = torch.sqrt(torch.mean((prop_preds - prop_targets) ** 2))
            rmses[prop_name] = rmse.item()
        else:
            rmses[prop_name] = float('nan')
    
    return avg_loss, rmses, (preds.numpy(), targets.numpy(), masks.numpy())


@torch.no_grad()
def predict(model, loader, device):
    """Generate predictions for test data."""
    model.eval()
    all_ids = []
    all_preds = []
    
    for data in tqdm(loader, desc="Predicting", leave=False):
        data = data.to(device)
        out = model(data)
        
        all_ids.extend(data.id)
        all_preds.append(out.cpu())
    
    predictions = torch.cat(all_preds, dim=0).numpy()
    
    return all_ids, predictions

print("Training functions defined!")

## 7. Competition Metrics

Implement the competition's weighted MAE metric.

In [ ]:
def weighted_mae(predictions, targets, masks=None):
    """Calculate the weighted Mean Absolute Error (wMAE) as defined in the competition."""
    # Convert to numpy if needed
    if torch.is_tensor(predictions):
        predictions = predictions.detach().cpu().numpy()
    if torch.is_tensor(targets):
        targets = targets.detach().cpu().numpy()
    if masks is not None and torch.is_tensor(masks):
        masks = masks.detach().cpu().numpy()
    
    # If no masks provided, assume all values are present
    if masks is None:
        masks = np.ones_like(predictions)
    
    # Calculate weights for each property
    weights = []
    total_samples = predictions.shape[0]
    
    for j in range(5):  # 5 properties
        # Count non-missing values for this property
        N_j = np.sum(masks[:, j])
        
        if N_j == 0:
            weights.append(0.0)
            continue
        
        # Calculate range R_j for this property (using only non-missing values)
        valid_mask = masks[:, j] == 1
        if np.sum(valid_mask) > 1:
            property_values = targets[valid_mask, j]
            R_j = np.max(property_values) - np.min(property_values)
            if R_j == 0:
                R_j = 1.0  # Avoid division by zero
        else:
            R_j = 1.0
        
        # Calculate weight: w_j = (5 / sqrt(N_j)) / R_j
        w_j = (5.0 / np.sqrt(N_j)) / R_j
        weights.append(w_j)
    
    # Normalize weights so they sum to 5
    weights = np.array(weights)
    if np.sum(weights) > 0:
        weights = weights * (5.0 / np.sum(weights))
    
    # Calculate weighted MAE
    total_weighted_error = 0.0
    total_count = 0
    
    for i in range(total_samples):
        for j in range(5):
            if masks[i, j] == 1:  # Only consider non-missing values
                error = abs(predictions[i, j] - targets[i, j])
                weighted_error = weights[j] * error
                total_weighted_error += weighted_error
                total_count += 1
    
    if total_count == 0:
        return float('inf')
    
    wmae = total_weighted_error / total_count
    return wmae


def print_competition_metrics(predictions, targets, masks=None):
    """Print comprehensive metrics in a nice format."""
    wmae = weighted_mae(predictions, targets, masks)
    
    print("\n" + "="*60)
    print("COMPETITION METRICS")
    print("="*60)
    print(f"Weighted MAE (Competition Metric): {wmae:.6f}")
    
    # Per-property metrics
    if masks is None:
        masks = np.ones_like(predictions)
    
    print("\nPer-Property Metrics:")
    print("-"*60)
    print(f"{'Property':<10} {'Count':<8} {'MAE':<12} {'RMSE':<12} {'Range':<12}")
    print("-"*60)
    
    for j, prop_name in enumerate(CONFIG.TARGET_PROPERTIES):
        valid_mask = masks[:, j] == 1
        count = np.sum(valid_mask)
        
        if count > 0:
            pred_vals = predictions[valid_mask, j]
            true_vals = targets[valid_mask, j]
            
            mae = np.mean(np.abs(pred_vals - true_vals))
            rmse = np.sqrt(np.mean((pred_vals - true_vals) ** 2))
            prop_range = np.max(true_vals) - np.min(true_vals) if len(true_vals) > 1 else 0.0
            
            print(f"{prop_name:<10} {count:<8} {mae:<12.6f} {rmse:<12.6f} {prop_range:<12.6f}")
        else:
            print(f"{prop_name:<10} {count:<8} {'N/A':<12} {'N/A':<12} {'N/A':<12}")
    
    print("="*60)

print("Competition metrics functions defined!")

## 8. Data Preparation and Training Loop

Prepare the datasets and train the model.

In [ ]:
# Prepare datasets
def prepare_datasets(train_df, test_df, val_split=0.2):
    """Prepare datasets for training, validation, and testing."""
    # Create datasets
    full_train_dataset = PolymerDataset(train_df, is_test=False)
    test_dataset = PolymerDataset(test_df, is_test=True)
    
    # Filter out invalid SMILES from training data
    valid_indices = []
    print("Filtering invalid SMILES...")
    for i in tqdm(range(len(full_train_dataset))):
        data = full_train_dataset.get(i)
        if data is not None:
            valid_indices.append(i)
    
    if len(valid_indices) != len(full_train_dataset):
        print(f"Warning: Filtered out {len(full_train_dataset) - len(valid_indices)} invalid SMILES from training data.")
    
    # Create clean training dataset
    train_df_clean = train_df.iloc[valid_indices].reset_index(drop=True)
    clean_train_dataset = PolymerDataset(train_df_clean, is_test=False)
    
    # Split training data into train and validation
    train_size = int((1.0 - val_split) * len(clean_train_dataset))
    val_size = len(clean_train_dataset) - train_size
    train_dataset, val_dataset = random_split(clean_train_dataset, [train_size, val_size])
    
    print(f"\nDataset sizes:")
    print(f"  Training: {len(train_dataset)}")
    print(f"  Validation: {len(val_dataset)}")
    print(f"  Test: {len(test_dataset)}")
    
    return train_dataset, val_dataset, test_dataset

# Prepare datasets
train_dataset, val_dataset, test_dataset = prepare_datasets(
    train_df, test_df, val_split=CONFIG.VAL_SPLIT_FRACTION
)

In [ ]:
# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=CONFIG.BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=CONFIG.BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=CONFIG.BATCH_SIZE, shuffle=False)

print(f"Data loaders created:")
print(f"  Train batches: {len(train_loader)}")
print(f"  Val batches: {len(val_loader)}")
print(f"  Test batches: {len(test_loader)}")

# Get feature dimensions
first_data = train_dataset[0]
num_atom_features = first_data.num_atom_features
print(f"  Atom features: {num_atom_features}")

In [ ]:
# Initialize model, optimizer, and scheduler
model = PolymerGCN(
    num_atom_features=num_atom_features,
    hidden_channels=CONFIG.HIDDEN_CHANNELS,
    num_gcn_layers=CONFIG.NUM_GCN_LAYERS
).to(CONFIG.DEVICE)

optimizer = torch.optim.Adam(
    model.parameters(),
    lr=CONFIG.LEARNING_RATE,
    weight_decay=CONFIG.WEIGHT_DECAY
)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=5
)

print(f"Model initialized:")
print(f"  Parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"  Device: {CONFIG.DEVICE}")

In [ ]:
# Training loop
print(f"Starting training for {CONFIG.NUM_EPOCHS} epochs...")
print("="*80)

# Training history
history = {
    'train_losses': [],
    'val_losses': [],
    'val_rmses': [],
    'learning_rates': [],
    'wmae_scores': []
}

best_val_loss = float('inf')
best_wmae = float('inf')
patience_counter = 0
early_stopping_patience = 15

for epoch in range(1, CONFIG.NUM_EPOCHS + 1):
    # Train
    train_loss = train_one_epoch(model, train_loader, optimizer, CONFIG.DEVICE)
    history['train_losses'].append(train_loss)
    
    # Validate
    val_loss, val_rmses, (val_preds, val_targets, val_masks) = evaluate(model, val_loader, CONFIG.DEVICE)
    history['val_losses'].append(val_loss)
    history['val_rmses'].append(val_rmses)
    history['learning_rates'].append(optimizer.param_groups[0]['lr'])
    
    # Calculate competition metric
    wmae = weighted_mae(val_preds, val_targets, val_masks)
    history['wmae_scores'].append(wmae)
    
    # Update learning rate
    scheduler.step(val_loss)
    
    # Print progress
    rmse_str = ", ".join([f"{k}: {v:.4f}" for k, v in val_rmses.items() if not np.isnan(v)])
    print(f"Epoch {epoch:3d} | Train: {train_loss:.4f} | Val: {val_loss:.4f} | wMAE: {wmae:.6f} | LR: {optimizer.param_groups[0]['lr']:.6f}")
    print(f"         RMSE: {rmse_str}")
    
    # Save best model
    if wmae < best_wmae:
        best_wmae = wmae
        best_val_loss = val_loss
        patience_counter = 0
        
        # Save model state
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_loss': val_loss,
            'wmae': wmae,
            'val_rmses': val_rmses,
        }, 'best_model.pt')
        
        print(f"         -> New best wMAE! Model saved.")
    else:
        patience_counter += 1
    
    # Early stopping
    if patience_counter >= early_stopping_patience:
        print(f"\nEarly stopping triggered after {patience_counter} epochs without improvement.")
        break
    
    print("-" * 80)

print(f"\nTraining complete!")
print(f"Best validation wMAE: {best_wmae:.6f}")
print(f"Best validation loss: {best_val_loss:.4f}")

In [ ]:
# Plot training history
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Training and validation loss
axes[0, 0].plot(history['train_losses'], label='Train Loss', alpha=0.8)
axes[0, 0].plot(history['val_losses'], label='Val Loss', alpha=0.8)
axes[0, 0].set_title('Training and Validation Loss')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Competition metric (wMAE)
axes[0, 1].plot(history['wmae_scores'], label='wMAE', color='red', alpha=0.8)
axes[0, 1].set_title('Competition Metric (wMAE)')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('wMAE')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Learning rate
axes[1, 0].plot(history['learning_rates'], label='Learning Rate', color='green', alpha=0.8)
axes[1, 0].set_title('Learning Rate Schedule')
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('Learning Rate')
axes[1, 0].set_yscale('log')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# RMSE by property
for prop in CONFIG.TARGET_PROPERTIES:
    rmse_values = [rmse_dict.get(prop, np.nan) for rmse_dict in history['val_rmses']]
    # Only plot if we have valid values
    if not all(np.isnan(rmse_values)):
        axes[1, 1].plot(rmse_values, label=prop, alpha=0.8)

axes[1, 1].set_title('Validation RMSE by Property')
axes[1, 1].set_xlabel('Epoch')
axes[1, 1].set_ylabel('RMSE')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 9. Evaluation and Submission

Load the best model and generate predictions for the test set.

In [ ]:
# Load the best model
print("Loading best model...")
checkpoint = torch.load('best_model.pt', map_location=CONFIG.DEVICE, weights_only=False)
model.load_state_dict(checkpoint['model_state_dict'])

print(f"Best model loaded:")
print(f"  Epoch: {checkpoint['epoch']}")
print(f"  Validation Loss: {checkpoint['val_loss']:.4f}")
print(f"  Competition wMAE: {checkpoint['wmae']:.6f}")

# Final evaluation on validation set
print("\nFinal validation evaluation:")
val_loss, val_rmses, (val_preds, val_targets, val_masks) = evaluate(model, val_loader, CONFIG.DEVICE)
print_competition_metrics(val_preds, val_targets, val_masks)

In [ ]:
# Generate predictions for test set
print("Generating predictions for test set...")
test_ids, test_predictions = predict(model, test_loader, CONFIG.DEVICE)

# Create submission DataFrame
submission_df = pd.DataFrame({
    'id': [int(id_val) if hasattr(id_val, 'item') else int(id_val) for id_val in test_ids],
    'Tg': test_predictions[:, 0],
    'FFV': test_predictions[:, 1],
    'Tc': test_predictions[:, 2],
    'Density': test_predictions[:, 3],
    'Rg': test_predictions[:, 4]
})

# Save submission
submission_filename = f"submission_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
submission_df.to_csv(submission_filename, index=False)

print(f"\nSubmission saved to: {submission_filename}")
print(f"Submission shape: {submission_df.shape}")
print("\nSample predictions:")
display(submission_df.head())

# Verify submission format
expected_columns = ['id', 'Tg', 'FFV', 'Tc', 'Density', 'Rg']
if list(submission_df.columns) == expected_columns:
    print("✅ Submission format is correct!")
else:
    print("❌ Submission format is incorrect!")
    print(f"Expected: {expected_columns}")
    print(f"Got: {list(submission_df.columns)}")

## 10. Results and Analysis

Analyze the model performance and predictions.

In [ ]:
# Analyze prediction distributions
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

for i, prop in enumerate(CONFIG.TARGET_PROPERTIES):
    # Get validation predictions and targets for this property
    prop_mask = val_masks[:, i] == 1
    
    if np.sum(prop_mask) > 0:
        pred_vals = val_preds[prop_mask, i]
        true_vals = val_targets[prop_mask, i]
        
        # Scatter plot: predictions vs targets
        axes[i].scatter(true_vals, pred_vals, alpha=0.6, s=20)
        
        # Perfect prediction line
        min_val = min(true_vals.min(), pred_vals.min())
        max_val = max(true_vals.max(), pred_vals.max())
        axes[i].plot([min_val, max_val], [min_val, max_val], 'r--', alpha=0.8, label='Perfect Prediction')
        
        # Calculate R²
        r2 = 1 - np.sum((true_vals - pred_vals)**2) / np.sum((true_vals - np.mean(true_vals))**2)
        
        axes[i].set_title(f'{prop} (n={len(pred_vals)}, R²={r2:.3f})')
        axes[i].set_xlabel('True Values')
        axes[i].set_ylabel('Predicted Values')
        axes[i].legend()
        axes[i].grid(True, alpha=0.3)
    else:
        axes[i].text(0.5, 0.5, f'No data for {prop}', 
                    horizontalalignment='center', verticalalignment='center',
                    transform=axes[i].transAxes, fontsize=14)
        axes[i].set_title(f'{prop} (No Data)')

# Remove empty subplot
fig.delaxes(axes[5])
plt.tight_layout()
plt.show()

In [ ]:
# Analyze test predictions
print("Test Prediction Statistics:")
print("=" * 50)

test_stats = submission_df[CONFIG.TARGET_PROPERTIES].describe()
display(test_stats)

# Plot test prediction distributions
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, prop in enumerate(CONFIG.TARGET_PROPERTIES):
    values = submission_df[prop]
    
    axes[i].hist(values, bins=20, alpha=0.7, edgecolor='black')
    axes[i].set_title(f'{prop} Test Predictions\n(mean={values.mean():.3f}, std={values.std():.3f})')
    axes[i].set_xlabel(prop)
    axes[i].set_ylabel('Frequency')
    axes[i].grid(True, alpha=0.3)

# Remove empty subplot
fig.delaxes(axes[5])
plt.tight_layout()
plt.show()

In [ ]:
# Summary
print("\n" + "="*80)
print("NEURIPS OPEN POLYMER PREDICTION 2025 - SOLUTION SUMMARY")
print("="*80)
print(f"Model Architecture: Graph Convolutional Network")
print(f"  - Input features: {num_atom_features} atom features")
print(f"  - Hidden channels: {CONFIG.HIDDEN_CHANNELS}")
print(f"  - GCN layers: {CONFIG.NUM_GCN_LAYERS}")
print(f"  - Output: 5 polymer properties")
print(f"  - Parameters: {sum(p.numel() for p in model.parameters()):,}")

print(f"\nTraining Configuration:")
print(f"  - Training samples: {len(train_dataset):,}")
print(f"  - Validation samples: {len(val_dataset):,}")
print(f"  - Test samples: {len(test_dataset):,}")
print(f"  - Batch size: {CONFIG.BATCH_SIZE}")
print(f"  - Learning rate: {CONFIG.LEARNING_RATE}")
print(f"  - Epochs trained: {len(history['train_losses'])}")

print(f"\nBest Performance:")
print(f"  - Validation Loss: {checkpoint['val_loss']:.4f}")
print(f"  - Competition wMAE: {checkpoint['wmae']:.6f}")

print(f"\nSubmission:")
print(f"  - File: {submission_filename}")
print(f"  - Format: ✅ Correct")
print(f"  - Ready for upload: ✅ Yes")

print("\nKey Features:")
print("  ✅ Multi-target prediction (5 properties simultaneously)")
print("  ✅ Missing value handling with binary masks")
print("  ✅ Competition metric implementation (weighted MAE)")
print("  ✅ Advanced molecular featurization (26 atom features)")
print("  ✅ Early stopping and learning rate scheduling")
print("  ✅ Comprehensive evaluation and visualization")

print("="*80)
print("🎉 SOLUTION COMPLETE - READY FOR SUBMISSION! 🎉")
print("="*80)